# Comparison with Julia Model

In [37]:
using F16Model, OrdinaryDiffEq


In [38]:

# Define state vector
# -------------------
d2r = pi/180;
npos = 0; # ft
epos = 0; # ft
alt = 10000; # should be in between 5000 ft and 100000 ft
phi = 0;   # rad -- Roll
theta = 0; # rad -- Pitch
psi = 0;   # rad -- Yaw
Vt = 300;  # ft/s -- Total velocity
alp = 0;   # rad -- Angle of attack
bet = 0;   # rad -- Side slip angle 
p = 0;     # rad/s -- Roll rate
q = 0;     # rad/s -- Pitch rate
r = 0;     # rad/s -- Yaw rate

x0 = [npos,epos,alt,phi,theta,psi,Vt,alp,bet,p,q,r];

# Define control vector
# ---------------------
T = 9000; # Thrust lbs
dele = 2; # deg elevator angle
dail = 2; # deg aileron angle
drud = 2; # deg rudder angle
dlef = 2; # deg leading edge flap angle
u0 = [T,dele,dail,drud,dlef];

In [39]:
tspan = (0.0,10);
xdot = zeros(12)

nlplant! = (xdot,x,p,t) -> F16Model.Dynamics!(xdot,x,u0)

prob = ODEProblem(nlplant!,x0,tspan, saveat=0.01);
sol = solve(prob);

In [40]:
xout = hcat(sol.u...)
tout = sol.t

using MAT
file = matopen("F16_Julia_Dump.mat","w")
write(file, "xout", xout)
write(file, "tout", tout)
close(file)

In [41]:
h0 = 10000; # ft
Vt0 = 500;   # ft/s

# Stead-Level Flight
xbar, ubar, status, prob = F16Model.Trim(h0,Vt0,γ=0, ψdot=0, ϕ=(0,1), ψ=(0,1), β=(0,1), p=(0,1), q=(0,1), r=(0,1));
xdot_trim = F16Model.Dynamics(xbar,ubar); # Should be close to zero.

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:       36

Total number of variables............................:        8
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9999900e-03 1.45e+02 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [42]:
using PrettyTables

# Check Trim Values
xuBounds = F16Model.StateAndControlBounds();
states = ["npos","epos","h","phi","theta","psi","V","alpha","beta","p","q","r"];
statesDer = ["Dnpos","Depos","Dh","Dphi","Dtheta","Dpsi","DV","Dalpha","Ddbeta","Dp","Dq","Dr"];
controls = ["T","ele","ail","rud","lef"];

xmin = vcat([-Inf;-Inf],xuBounds[1:10,1]); xmax = vcat([Inf;Inf], xuBounds[1:10,2]);

data = hcat(states,xdot_trim,xmin,xbar,xmax)
pretty_table(header=["states","xdot","xmin","xbar","xmax"],data);
pretty_table(header=["controls","umin","ubar","umax"],hcat(controls,xuBounds[11:end,1],ubar,xuBounds[11:end,2]));
     

┌────────┬──────────────┬───────────┬───────────┬──────────┐
│ states │         xdot │      xmin │      xbar │     xmax │
├────────┼──────────────┼───────────┼───────────┼──────────┤
│   npos │        500.0 │      -Inf │       0.0 │      Inf │
│   epos │          0.0 │      -Inf │       0.0 │      Inf │
│      h │   3.02269e-5 │    5000.0 │   10000.0 │  40000.0 │
│    phi │          0.0 │   -1.5708 │       0.0 │   1.5708 │
│  theta │          0.0 │   -1.5708 │ 0.0590342 │   1.5708 │
│    psi │          0.0 │   -1.5708 │       0.0 │   1.5708 │
│      V │    1.8945e-9 │     300.0 │     500.0 │    900.0 │
│  alpha │   8.79867e-6 │ -0.349066 │ 0.0590341 │ 0.785398 │
│   beta │ -0.000701234 │ -0.523599 │       0.0 │ 0.523599 │
│      p │   6.75021e-8 │      -Inf │       0.0 │      Inf │
│      q │  -2.15363e-7 │      -Inf │       0.0 │      Inf │
│      r │  -9.08611e-6 │      -Inf │       0.0 │      Inf │
└────────┴──────────────┴───────────┴───────────┴──────────┘
┌──────────┬────────┬───

In [43]:
A, B = F16Model.Linearize(xbar,ubar);

# Pull out longitudinal dynamics -- states:(theta,V,alpha,q), control:(T,ele)
ix = [5,7,8,11]; iu = [1,2]; # 4 states, 3 control system.
longiA = A[ix,ix];
longiB = B[ix,iu];

# Show system
pretty_table(header = vcat("xdotLongi",states[ix],controls[iu]),hcat(statesDer[ix],longiA,longiB))

# Eigen values 
using LinearAlgebra
ev = eigen(longiA).values;
pretty_table(header=["Eigen values"],ev)

# plot(real(ev),imag(ev),seriestype=:scatter,label="Eigen values")

┌───────────┬─────────────┬──────────────┬───────────┬──────────┬────────────┬─────────────┐
│ xdotLongi │       theta │            V │     alpha │        q │          T │         ele │
├───────────┼─────────────┼──────────────┼───────────┼──────────┼────────────┼─────────────┤
│    Dtheta │         0.0 │          0.0 │       0.0 │      1.0 │        0.0 │         0.0 │
│        DV │      -32.17 │   -0.0118591 │  -2.58453 │ -1.28295 │ 0.00156727 │   0.0662367 │
│    Dalpha │ -3.88959e-9 │ -0.000255941 │ -0.782055 │ 0.929171 │ -1.8526e-7 │ -0.00172694 │
│        Dq │         0.0 │ -8.61453e-10 │ -0.455764 │ -1.01533 │        0.0 │   -0.140354 │
└───────────┴─────────────┴──────────────┴───────────┴──────────┴────────────┴─────────────┘
┌─────────────────────────┐
│            Eigen values │
├─────────────────────────┤
│    -0.901194-0.640782im │
│    -0.901194+0.640782im │
│ -0.00342886-0.0552919im │
│ -0.00342886+0.0552919im │
└─────────────────────────┘


In [44]:
longiA

4×4 Matrix{Float64}:
   0.0          0.0           0.0        1.0
 -32.17        -0.0118591    -2.58453   -1.28295
  -3.88959e-9  -0.000255941  -0.782055   0.929171
   0.0         -8.61453e-10  -0.455764  -1.01533

In [45]:
0.785398*180/pi


44.99999063801584